## Testing General Embedding Extraction Model

Model's Description: https://huggingface.co/Alibaba-NLP/gte-multilingual-base

In [4]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
from sentence_transformers import SentenceTransformer

model_name_or_path="Alibaba-NLP/gte-multilingual-base"
model = SentenceTransformer(model_name_or_path, trust_remote_code=True, device=device)

input_texts = [
    "what is the capital of China?",
    "Who created the Qwen Model?",
    "What is qwen2.5 1.5B?",
    "I start to wonder which LLMs are the best on the market today.",
]
embeddings = model.encode(input_texts, normalize_embeddings=True) # embeddings.shape (4, 768)

Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The 2nd and 3rd texts both have the word "qwen" in them, leading to higher similarity score.

In [6]:
scores = model.similarity(embeddings[:], embeddings[:])
scores

tensor([[1.0000, 0.3980, 0.3863, 0.3008],
        [0.3980, 1.0000, 0.7581, 0.4808],
        [0.3863, 0.7581, 1.0000, 0.4447],
        [0.3008, 0.4808, 0.4447, 1.0000]])

##### On case sensitivity 
The 2nd text contains "Qwen" and 3rd text has "qwen" differing on the case of the the character "q".
<br/>
To see the affect of the case of the character of a word, we changed the 3rd text "qwen" to "Qwen", and re-evaluate the score again.
<br/>
The similarity score went from 0.758 to 0.790. (i.e. the two texts' similarity score got stronger)

In [8]:
input_texts = [
    "Who created the Qwen Model?",
    "What is Qwen2.5 1.5B?",
]
embeddings = model.encode(input_texts, normalize_embeddings=True)

scores = model.similarity(embeddings[:], embeddings[:])
scores

tensor([[1.0000, 0.7903],
        [0.7903, 1.0000]])

##### On Longer text
<em>Question</em>: "How good is the embedding model on longer sentences or paragraph". <br />
<em>Setting</em>: Prompted ChatGPT "Generate 3 short paragraphs having similar meanings but contains no same words using a theme of country life in the 1960s." <br />
<em>Findings</em>: The embedding model is doing a pretty good job. (with the min similarity score = 0.779)

In [9]:
input_texts = [
    "In the early morning haze, roosters crowed across the rolling fields while dew clung to every blade of grass. Families began their chores before the sun fully rose, tending livestock and gathering eggs from wooden coops. Life moved slowly, in rhythm with the seasons and the land’s quiet demands.",
    "Sunrise brought the sound of hens clucking and tractors humming in the distance. Barn doors creaked open as folks stepped out in boots, ready to milk cows and repair fence posts. Each day was grounded in simplicity, shaped by weather, earth, and honest sweat.",
    "At dawn, the scent of hay mixed with the aroma of fresh bread from old stoves. Children ran barefoot through orchards while elders watered gardens and patched sheds. Community ties were strong, forged through shared labor and humble routines."
]
embeddings = model.encode(input_texts, normalize_embeddings=True)

scores = model.similarity(embeddings[:], embeddings[:])
scores

tensor([[1.0000, 0.8245, 0.7790],
        [0.8245, 1.0000, 0.8041],
        [0.7790, 0.8041, 1.0000]])

##### On Non-English Text (Korean)
<em>Question</em>: "How good is the embedding model on longer sentences or paragraph on other languages beside English". <br />
<em>Setting</em>: Prompted ChatGPT "Generate 3 short paragraphs in Korean having similar meanings but contains no same words using a theme of country life in the 1960s." <br />
<em>Findings</em>: The embedding model is doing a pretty good job. (with the min similarity score = 0.629)

In [10]:
input_texts = [
    "1960년대 전원에서는 아침이면 닭 울음소리에 하루가 시작되었다. 뒷산에서 나무를 하거나 논에서 일하며 가족끼리 생계를 꾸렸다. 소박한 식사와 함께 따뜻한 정이 오갔던 시절이었다.",
    "그 시절 농촌 마을은 새벽이면 종달새 소리에 잠이 깼고, 마당에서는 아이들이 흙장난을 하며 놀았다. 땀 흘려 일한 뒤 먹는 된장국 한 그릇이 그렇게도 꿀맛 같았다. 모두가 서로를 도우며 살아갔다.",
    "오래전 시골집에서는 해돋이 무렵부터 사람들이 들판으로 나섰다. 물레방아 소리와 개 짖는 소리가 어우러진 조용한 풍경 속에서, 이웃 간 우애가 깊이 배어 있었다. 여유는 없었지만 마음은 풍족했다.",
]
embeddings = model.encode(input_texts, normalize_embeddings=True)

scores = model.similarity(embeddings[:], embeddings[:])
scores

tensor([[1.0000, 0.7043, 0.6289],
        [0.7043, 1.0000, 0.6963],
        [0.6289, 0.6963, 1.0000]])

##### Insight of similarity score of texts having different themes.
<em>Question</em>: "How good is the embedding model on longer text that are completely different in categories (Politics, Entertainments, Technologies, etc.)". <br />
<em>Setting</em>: Extract 3 paragraphs from 3 different news articles with categories, Heath, Technology, Economy.  <br />
<em>Findings</em>: Texts from different categories shows max similarity score of 0.42 (in this setting).

In [11]:
input_texts = [
    "A second unvaccinated child has died from measles in Texas, as the United States confronts its worst outbreak of the disease in more than 30 years. The school-aged child, with no underlying conditions, died from complications while hospitalised, according to the University Medical Center (UMC) Health System in Lubbock, a leading regional hospital and teaching institution. The death follows another fatal pediatric case in Texas in February and an adult death in neighbouring New Mexico in early March.",
    "Last month, the Pixel 9 gained real-time Scam Detection, and those warnings are now coming to your paired Pixel Watch 2 and 3: “So if you’re on the go, you can be alerted to a scam call without needing to take out your phone.” The Pixel 9, 9 Pro, 9 Pro XL, and 9 Pro Fold leverage Gemini Nano with Multimodality to detect suspicious patterns commonly used by scammers. Your phone will vibrate and make noise to get you to see the on-screen “Likely scam” warning. This works on-device and in real-time.",
    "The Commerce Ministry said the U.S.‘s imposition of “so-called ‘reciprocal tariffs’” on China is “completely groundless and is a typical unilateral bullying practice.” China, the world’s second-largest economy, has taken retaliatory tariffs and the ministry hinted in its latest statement that more may be coming. “The countermeasures China has taken are aimed at safeguarding its sovereignty, security and development interests, and maintaining the normal international trade order. They are completely legitimate,” the ministry said. “The U.S. threat to escalate tariffs on China is a mistake on top of a mistake and once again exposes the blackmailing nature of the US. China will never accept this. If the US insists on its own way, China will fight to the end,” it added."
]
embeddings = model.encode(input_texts, normalize_embeddings=True)

scores = model.similarity(embeddings[:], embeddings[:])
scores

tensor([[1.0000, 0.4058, 0.4236],
        [0.4058, 1.0000, 0.4407],
        [0.4236, 0.4407, 1.0000]])

##### Insight of similarity score of texts having similar themes.
<em>Question</em>: "How good is the embedding model on longer text that are similar in categories ("Politics" in this case)". <br />
<em>Setting</em>: Extract 3 paragraphs from 3 policical news articles but from different times (2008, 2015 & 2024).  <br />
<em>Findings</em>: Max similarity score of 0.44 (in this setting). I'd say the domain or theme of the text doesn't affect the similarity score of the text itself much rather than the actual content of the text.

In [12]:
input_texts = [
    "Police estimated that 50,000 people jammed a plaza in front of Seoul's City Hall and an adjacent 14-lane boulevard, holding candles and anti-government signs. The protesters, including Catholic priests and Buddhist monks, sang songs and chanted slogans. About 20,000 riot police were deployed around the site, but there were no immediate reports of clashes. It was the second-largest rally in a series of near-daily protests held for the past two months prompted by concerns over the safety of American beef imports. A rally on June 10 drew a crowd estimated by police at about 80,000.",
    "Donald Trump has dominated the Republican field so far, even though he has relied largely on free media attention to get the word out about his campaign. By contrast, Jeb Bush's campaign and his allies have spent more than $40 million so far -- only to watch the former governor flounder in the single digits in polls. \"He hasn't spent $40 million. He's wasted $40 million. There's a big difference,\" Trump said on Tuesday, announcing that he will finally start investing in TV ads.",
    "After two sessions of sell-offs that wiped away over $5.4 trillion in market value, Dow futures were down 1,500 points, or 4%. S&P 500 futures plunged 4.3%, while Nasdaq futures tumbled 4.7%. Stocks were set to open sharply lower Monday, putting the S&P 500 on the precipice of a bear market — a decline of 20% from its peak and an ominous sign for investors and perhaps the broader economy."
]
embeddings = model.encode(input_texts, normalize_embeddings=True)

scores = model.similarity(embeddings[:], embeddings[:])
scores

tensor([[1.0000, 0.3954, 0.4432],
        [0.3954, 1.0000, 0.4373],
        [0.4432, 0.4373, 1.0000]])

##### On multilingual text
> Same text content but different languages.

<em>Question</em>: "How good is the embedding model on the same text but translated to differnet language.. <br />
<em>Setting</em>: Asked ChatGPT to translate one of the text of the above setting from English to Korean and Chinese, and place the texts for embedding extraction.  <br />
<em>Findings</em>: Min similarity score of 0.848 (in this setting).

In [26]:
input_texts = [
    "Police estimated that 50,000 people jammed a plaza in front of Seoul's City Hall and an adjacent 14-lane boulevard, holding candles and anti-government signs. The protesters, including Catholic priests and Buddhist monks, sang songs and chanted slogans. About 20,000 riot police were deployed around the site, but there were no immediate reports of clashes. It was the second-largest rally in a series of near-daily protests held for the past two months prompted by concerns over the safety of American beef imports. A rally on June 10 drew a crowd estimated by police at about 80,000.",
    "경찰에 따르면, 약 5만 명의 사람들이 서울 시청 앞 광장과 그에 인접한 14차선 도로를 가득 메우고 촛불과 반정부 구호가 적힌 팻말을 들고 모였다. 시위대에는 가톨릭 신부들과 불교 승려들도 포함되어 있었으며, 이들은 노래를 부르고 구호를 외쳤다. 약 2만 명의 전투경찰이 현장 주변에 배치되었지만, 즉각적인 충돌 보고는 없었다. 이번 시위는 지난 두 달간 거의 매일 열리고 있는 시위 중 두 번째로 큰 규모로, 미국산 쇠고기의 안전성에 대한 우려로 촉발된 것이다. 지난 6월 10일의 시위에는 경찰 추산 약 8만 명이 모였다.",
    "据警方估计，大约5万人聚集在首尔市政厅前的广场以及相邻的14车道大道上，手持蜡烛和反政府标语。抗议者中包括天主教神父和佛教僧侣，他们唱着歌、呼喊口号。大约2万名防暴警察被部署在现场周围，但没有立即发生冲突的报告。这是过去两个月几乎每天举行的抗议活动中，规模第二大的集会。此前6月10日的一次抗议中，警方估计约有8万人参加"
]
embeddings = model.encode(input_texts, normalize_embeddings=True)

scores = model.similarity(embeddings[:], embeddings[:])
scores

tensor([[1.0000, 0.8600, 0.8488],
        [0.8600, 1.0000, 0.8630],
        [0.8488, 0.8630, 1.0000]])